<a href="https://colab.research.google.com/github/madelineschwarz/googEarthEng_misc/blob/main/Sample_3DEP_gcps.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<table class="ee-notebook-buttons" align="left"><td>
<a target="_blank"  href="http://colab.research.google.com/github/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb">
    <img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a>
</td><td>
<a target="_blank"  href="https://github.com/google/earthengine-community/blob/master/guides/linked/ee-api-colab-setup.ipynb"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a></td></table>

### Import the API

Run the following cell to import the API into your session.

In [ ]:
# To use the Google Collab T4 GPU:
#import tensorflow as tf
# Check GPU availability
#print("GPU Available:", tf.config.experimental.list_physical_devices("GPU"))

GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
import ee
import geopandas
import geemap
import geopandas as gpd
from shapely.geometry import box
from shapely.geometry import Point
import numpy as np
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='iron-atom-398503')

In [ ]:
# function to sample 3DEP data using gdf points
def sample_3dep_data(gdf):
    # Load the 3DEP DEM ImageCollection
    dem_collection = ee.ImageCollection("USGS/3DEP/1m")

    # Convert the GeoDataFrame to an Earth Engine FeatureCollection
    ee_points = ee.FeatureCollection(gdf)

    # Define a function to sample elevation, latitude, and longitude values at the points
    def sample_at_points(image):
        sampled_points = image.sample(
            region=ee_points.geometry(),
            scale=1,  # 1-meter scale for 1-meter DEM data
        )
        return sampled_points

    # Map the sampling function over the ImageCollection
    sampled_points_collection = dem_collection.map(sample_at_points)

    # Flatten the resulting FeatureCollection
    sampled_points_flat = sampled_points_collection.flatten()

    # Define a function to extract elevation, latitude, and longitude values
    def extract_3dep_data(feature):
        elevation = feature.get("elevation")
        return feature.set({
            "elevation": elevation  # Assign elevation to the 'altitude' attribute
        })

    # Apply the extraction function to each sampled point
    extracted_data = sampled_points_flat.map(extract_3dep_data)

    # Convert the result back to a GeoDataFrame
    extracted_gdf = gpd.GeoDataFrame.from_features(extracted_data.getInfo())

    return extracted_gdf

In [ ]:
#"EPSG:4326"  # WGS84


markers = 'small_cala_markers.shp'
points_gdf = gpd.read_file(markers)
# Convert GeoDataFrame to Earth Engine objects
ee_points = shp_to_ee(points_gdf)

result_gdf = sample_3dep_data(ee_points)

In [ ]:
# View the sampled elevation points
# Conversion from gdf to ee object lost the geometry, lat, long columns
print(result_gdf)
print(type(result_gdf))

   geometry   elevation
0      None  351.222595
1      None  305.486145
2      None  323.393433
3      None  324.583557
4      None  310.290283
..      ...         ...
94     None  239.964767
95     None  300.238403
96     None  282.850250
97     None  268.594360
98     None  223.753098

[99 rows x 2 columns]
<class 'geopandas.geodataframe.GeoDataFrame'>


In [ ]:
# Add Longitude & Latitude Columns to Result GDF
result_gdf['geometry'] = points_gdf['geometry']
result_gdf['latitude'] = points_gdf['latitude']
result_gdf['longitude'] = points_gdf['longitude']
result_gdf

,geometry,elevation,latitude,longitude
0,POINT (-121.27390 36.69001),351.222595,36.69001,-121.273900
1,POINT (-121.27086 36.69001),305.486145,36.69001,-121.270862
2,POINT (-121.26782 36.69001),323.393433,36.69001,-121.267824
3,POINT (-121.26479 36.69001),324.583557,36.69001,-121.264786
4,POINT (-121.26175 36.69001),310.290283,36.69001,-121.261748
...,...,...,...,...
94,POINT (-121.25567 36.71644),239.964767,36.71644,-121.255672
95,POINT (-121.25263 36.71644),300.238403,36.71644,-121.252634
96,POINT (-121.24960 36.71644),282.850250,36.71644,-121.249596
97,POINT (-121.24656 36.71644),268.594360,36.71644,-121.246558


In [ ]:
# Export Sampled points as a shapefile
